# BITSTAMP BTC/USD Trade data

In [71]:
## COIN CAP API
import requests
import pandas as pd
import numpy as np
import datetime
import os

# Yigit's functions
import sys
sys.path.append('../../Time_Series_Forecasting/TS_helpers/')
from dates_coinapi import make_all_time_indexes, make_integer_time_index

api_key = "B90AC06B-4BCB-4FC2-B3BA-CBBABCF63791"


# URL and headers for coin api account
url = 'https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history?'
headers = {'X-CoinAPI-Key' : api_key}


# Directory where data is read from and written to
data_dir = '../../Data/Coins/BTC_USD/Trade_Data/'

## Load existing(cumulative) data / Add new / Save all together csv

In [72]:
df_saved = pd.read_csv( data_dir + 'BITSTAMP_TRADE_SPOTBTC_last_saved_trade.csv' , index_col='uuid')
df_saved.shape
# df_new.tail(1).to_csv(data_dir + 'BITSTAMP_TRADE_SPOTBTC_last_saved_trade.csv')

(1, 19)

In [73]:
df_saved

,price,size,symbol_id,taker_side,time_coinapi,time_exchange,year,month,day,hour,minute,second,time_year,time_month,time_day,time_hour,time_minute,time_second,date
uuid,,,,,,,,,,,,,,,,,,,
06f5bf72-488a-41e5-8e09-7bd6f695ef73,748.02,0.01,BITSTAMP_SPOT_BTC_USD,SELL,2016-11-19T00:30:35.7033883Z,2016-11-19T00:30:35.0000000Z,2016,11,19,0,30,35,2016,201611,20161119,2016111900,201611190030,20161119003035,2016-11-19


##### TODO ADD DATE MANIPULATION FOR STARTING FROM NEXT DAY

In [125]:
# Manual day addition until above code fixed

first_day_of_new = df_saved['time_exchange'].max()
if first_day_of_new[-9:-1] != '.9999999':
    next_time = str(int(first_day_of_new[-8:-1])+1)
    first_day_of_new = first_day_of_new[:-8]+ "0"*int(len('0000000')-len(next_time)) + next_time + 'Z'
else:
    None
    
last_day_of_new = '2018-01-01'

In [126]:
print(first_day_of_new)
print(last_day_of_new)

2016-11-19T00:30:35.0000001Z
2018-01-01


#### Get JSON data for next day(s)

In [127]:
url = 'https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history?time_start='+ first_day_of_new + '&time_end=' + last_day_of_new + '&limit=100000'

# # url = 'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?period_id=1MIN&time_start=2016-01-01T00:00:00'

response = requests.get(url, headers=headers)

if response.content[2:7] != b'error':
    df_new = pd.read_json(response.content).set_index('uuid')
    print(df_new.shape)
else:
    print('Limit yok.')

Limit yok.


### Make integer indexes

In [60]:
df_saved.columns

df_new = make_all_time_indexes(df_new)

cycle year
cycle month
cycle day
cycle hour
cycle minute
cycle second
bin year
bin month
bin day
bin hour
bin minute
bin second


In [128]:
print(df_saved['time_exchange'].unique().max())
print(df_new['time_exchange'].unique().min())
print(df_new['time_exchange'].unique().max())

2016-11-19T00:30:35.0000000Z
2016-11-10T04:14:17.0000000Z
2016-11-19T00:30:35.0000000Z


In [62]:
# from datetime import datetime as dt

# dt.strptime('2016-04-10T22:02:50.73000Z', '%Y-%m-%dT%H:%M:%S.%fZ')

### Indirilen yeni verinin uygunluguna dair bazi checkler

In [63]:
df_new.shape[0] == df_new.index.nunique()

True

In [64]:
all(df_new.isna().mean() == 0)

True

In [65]:
df_both = df_saved.append(df_new)

In [66]:
df_both.shape[0] - df_saved.shape[0] == df_new.shape[0]

True

In [67]:
df_both.head()

,price,size,symbol_id,taker_side,time_coinapi,time_exchange,year,month,day,hour,minute,second,time_year,time_month,time_day,time_hour,time_minute,time_second,date
uuid,,,,,,,,,,,,,,,,,,,
a8daddb5-23f1-46a3-a63a-19304a3e1499,714.55,0.251000,BITSTAMP_SPOT_BTC_USD,BUY,2016-11-10T04:13:48.0711286Z,2016-11-10T04:13:47.0000000Z,2016,11,10,4,13,47,2016,201611,20161110,2016111004,201611100413,20161110041347,2016-11-10
62961fca-4818-427d-bbb8-7077d5d7fe0a,716.19,0.193893,BITSTAMP_SPOT_BTC_USD,BUY,2016-11-10T04:14:17.7898878Z,2016-11-10T04:14:17.0000000Z,2016,11,10,4,14,17,2016,201611,20161110,2016111004,201611100414,20161110041417,2016-11-10
6bc3ce5b-74b4-4651-948c-9a52e38093ea,716.21,0.757561,BITSTAMP_SPOT_BTC_USD,BUY,2016-11-10T04:14:17.9305174Z,2016-11-10T04:14:17.0000000Z,2016,11,10,4,14,17,2016,201611,20161110,2016111004,201611100414,20161110041417,2016-11-10
9a1306e8-4797-481e-bf72-55a8a45419a6,716.22,0.070000,BITSTAMP_SPOT_BTC_USD,BUY,2016-11-10T04:14:37.1805221Z,2016-11-10T04:14:36.0000000Z,2016,11,10,4,14,36,2016,201611,20161110,2016111004,201611100414,20161110041436,2016-11-10
64287d13-4e40-4422-a22a-3dfb31dbfe44,716.22,0.095000,BITSTAMP_SPOT_BTC_USD,BUY,2016-11-10T04:14:51.7117798Z,2016-11-10T04:14:51.0000000Z,2016,11,10,4,14,51,2016,201611,20161110,2016111004,201611100414,20161110041451,2016-11-10


### CSVye eklenenen yeni veriyi diske yaz

In [68]:
with open(data_dir + 'BITSTAMP_TRADE_SPOTBTC_cum.csv','a') as fd:
    df_new.to_csv(fd, header=False)

### Sadece son(zaman olarak) trade verisini ayri olarak kaydet
###### (sonraki sefer kaldigimiz yerden devam ederken, nerede kaldigimizi gormek icin butun veriyi hafizaya yuklemek zorunda kalmayalim diye)

In [70]:
df_new.tail(1).to_csv(data_dir + 'BITSTAMP_TRADE_SPOTBTC_last_saved_trade.csv')

In [206]:
# line = 0
# i = 0

# lines = []
# after = False
# with open(data_dir + 'BITSTAMP_TRADE_SPOTBTC_cum.csv','w') as fd:
#     while line is not None:
#         i = i+1
#         line = fd.readline()
#         print(line)
#         if line is not None:
            
#             if len(line.split(',')) == 39:
#                 after = True
#                 print(i)
#                 print(line.split(','))
#         if after:
#             print(line)
#     #     fd.readlines(999300)

        
# #         line_lens.append(len(fd.readline().split(',')))

In [205]:
# write_lines = []
# vals_line = lines[100].split(',')
# for line in lines:
#     valsinline = line.split(',')
#     if(len(valsinline)==39):
#         vals_line_1 = valsinline[:19]
#         vals_line_2 = valsinline[20:]
#         last_of_line_1 = valsinline[19][:-36]+'\n'
#         first_of_line_2 = valsinline[19][-36:]
# #         print(last_of_line_1, first_of_line_2)
#         vals_line_1.append(last_of_line_1+'\n')
#         vals_line_2.insert(0, first_of_line_2)
        
#         line_1 = ','.join(vals_line_1)
#         line_2 = ','.join(vals_line_2)
        
#         print(line_1)
#         for val in vals_line_1:
#             print(val)
        
#         print(line_2)
#         for val in vals_line_2:
#             print(val)
            
#         print(lines[100])
#         for val in vals_line:
#             print(val)
            
#         write_lines.append(line_1)
#         write_lines.append(line_2)
        
#     else:
#         write_lines.append(line)
        
        
        

In [151]:
print(line_39[0].split(',')[19][-36:], line_39[0].split(',')[19][:-36] )

19f3608c-096e-44e2-ad58-e015f2fc4d88 2016-07-09


In [150]:
# line_39[0].split(',')

In [172]:
with open(data_dir + 'BITSTAMP_TRADE_SPOTBTC_cum.csv','w') as fw:
    fw.writelines(write_lines)
    